# Selector Group Chat

SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY")

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [6]:
model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    model_info={
        "family": "mistral",
        "vision": True,
        "function_calling": True,
        "json_output": False,
        "structured_output": False
    }
)

1st Agent ??

In [7]:
planning_agent = AssistantAgent(
    name = 'PlanningAgent',
    description= 'An agent for planning tasks,this agent should be the first to engage when given a new task.',
    model_client=model_client,
    system_message='''
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are :
        WebSearchAgent : Searches for information.
        DataAnalystAgent : Performs calculations

    You only plan and delegate tasks - you do not exectue them yourself.

    When assigning tasks, use the below format:
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and end with "TERMINATE"
    ''',
)

2nd Agent ??

In [8]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [9]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

3rd Agent ??

In [10]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

In [11]:
data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.

    ''',
)

### Termination Conditions

In [12]:
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

## Define The First Way Of Termination Conditions - TextMentionTermination
text_mention_termination = TextMentionTermination("TERMINATE")

## Define The Second Way Of Termination Conditions - MaxMessageTermination
max_messages_termination = MaxMessageTermination(max_messages=6)

## Combine The Both Ways ??
combined_termination = text_mention_termination | max_messages_termination

In [13]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

Current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planning agent has assigned task before other agents start working.
Only Select one agent.
'''

In [14]:
'''
Try not to overload the selector prompt with too much information.


selector_prompt (str, optional) – The prompt template to use for selecting the next speaker. 

Available fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. 
1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. 
2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. 

3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.
'''

'\nTry not to overload the selector prompt with too much information.\n\n\nselector_prompt (str, optional) – The prompt template to use for selecting the next speaker. \n\nAvailable fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. \n1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. \n2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. \n\n3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.\n'

#### SelectorGroupChat
- Create The SelectorGroupChat Which Restrict The Team To Output Results With Specific Constrains

In [15]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True
)

In [16]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [17]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Gather data on the total rebounds of the player identified in the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (PlanningAgent) ----------
To calculate percentage change, we use the formula:

Percentage Change = [(New Value - Old Value) / Old Value] * 100

Calculate it for the identified player's rebounds.
---------- TextMessage (WebSearchAgent) ----------
I apologize, but I cannot perform data analysis or calculations as per my instructions. If you provide the necessary 

TaskResult(messages=[TextMessage(id='833d6798-09b5-4a03-92ca-13ca8c4d4117', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 22, 42, 38, 816488, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='e26c30c9-69b1-4c66-8dc6-1024f775f95f', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=173, completion_tokens=114), metadata={}, created_at=datetime.datetime(2025, 8, 4, 22, 42, 48, 777612, tzinfo=datetime.timezone.utc), content="1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent: Gather data on the total rebounds of the player identified in the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 

#### Custom Selector Function

In [18]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage

def selector_func(messages : Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if(messages[-1].source != planning_agent.name):
        return planning_agent.name
    return None

In [19]:
await selector_team.reset()

selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func
)

In [20]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.
2. DataAnalystAgent: Calculate the percentage change in total rebounds for the identified player between the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (PlanningAgent) ----------
 externe Tháng 61, 2024
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 season.
3. WebSearchAgent: Find the total rebounds for this player in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds between the 2007-2008 and 2008-200

TaskResult(messages=[TextMessage(id='3bdbac5b-afbe-429c-8bdc-fc41332640d3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 22, 48, 3, 682657, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='640eb46e-f2a7-4808-ad01-7368aa0c218c', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=173, completion_tokens=73), metadata={}, created_at=datetime.datetime(2025, 8, 4, 22, 48, 7, 462743, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. DataAnalystAgent: Calculate the percentage change in total rebounds for the identified player between the 2007-2008 and 2008-2009 seasons.', type='TextMessage'), TextMessage(id='add930e9-b3fb-4d82-bb4c-49030d160531'